In [ ]:
# =============================================================================
# ⚙️ CONFIGURATION CENTRALISÉE
# =============================================================================

import sys
import os
import time
from pathlib import Path

# =============================================================================
# 🌍 DÉTECTION AUTOMATIQUE DE L'ENVIRONNEMENT
# =============================================================================

def detect_environment():
    """Détecte automatiquement si on est sur Colab ou en local"""
    try:
        import google.colab
        return True
    except ImportError:
        return False

# =============================================================================
# 📋 CONFIGURATION PRINCIPALE
# =============================================================================

class Config:
    """Configuration centralisée du projet"""
    
    # 🌍 Environnement
    USING_COLAB = detect_environment()
    
    # 🎬 VIDÉO ET PROJET  
    VIDEO_NAME = "SD_13_06_2025_1_PdB_S1_T959s"  # ⚠️ MODIFIEZ ICI
    FRAME_INTERVAL = 3                            # ⚠️ MODIFIEZ ICI
    
    # 🎬 OPTIONS D'EXTRACTION
    EXTRACT_FRAMES = True                         # ⚠️ MODIFIEZ ICI
    FORCE_EXTRACTION = False                      # ⚠️ MODIFIEZ ICI
    
    # 🤖 SAM2 CONFIGURATION
    SAM2_MODEL = "sam2.1_hiera_l"                # ou "sam2.1_hiera_s" pour small
    SAM2_CHECKPOINT = "sam2.1_hiera_large.pt"    # ou "sam2.1_hiera_small.pt"
    
    # 🗂️ CHEMINS AUTOMATIQUES
    @property
    def videos_dir(self):
        return Path("./videos") if self.USING_COLAB else Path("../data/videos")
    
    @property 
    def checkpoint_path(self):
        base = "../../checkpoints" if self.USING_COLAB else "../checkpoints"
        return Path(base) / self.SAM2_CHECKPOINT
    
    @property
    def model_config_path(self):
        return f"configs/sam2.1/{self.SAM2_MODEL}.yaml"
    
    # 🎬 CHEMINS VIDÉO
    @property
    def video_path(self):
        return self.videos_dir / f"{self.VIDEO_NAME}.mp4"
    
    @property
    def config_path(self):
        return self.videos_dir / f"{self.VIDEO_NAME}_config.json"
    
    @property
    def output_dir(self):
        return self.videos_dir / "outputs" / self.VIDEO_NAME
    
    @property
    def frames_dir(self):
        return self.output_dir / "frames"
    
    @property
    def masks_dir(self):
        return self.output_dir / "masks"
    
    @property
    def output_video_path(self):
        return self.output_dir / f"{self.VIDEO_NAME}_annotated.mp4"
    
    @property
    def output_json_path(self):
        return self.output_dir / f"{self.VIDEO_NAME}_project.json"
    
    def setup_directories(self):
        """Crée tous les dossiers nécessaires (sans vérifier les fichiers)"""
        print("🏗️ Création de la structure de dossiers...")
        self.videos_dir.mkdir(parents=True, exist_ok=True)
        self.output_dir.mkdir(parents=True, exist_ok=True)
        self.frames_dir.mkdir(exist_ok=True)
        self.masks_dir.mkdir(exist_ok=True)
        print(f"   ✅ Dossiers créés dans: {self.output_dir}")
    
    def check_files_exist(self):
        """Vérifie si les fichiers requis existent (sans lever d'erreur)"""
        video_exists = self.video_path.exists()
        config_exists = self.config_path.exists()
        
        print(f"📄 Vérification des fichiers:")
        print(f"   🎬 Vidéo: {'✅' if video_exists else '❌'} {self.video_path}")
        print(f"   📄 Config: {'✅' if config_exists else '❌'} {self.config_path}")
        
        return video_exists and config_exists
    
    def wait_for_files(self, max_wait_minutes=10, check_interval=10):
        """Attend que les fichiers soient disponibles (utile pour Colab)"""
        if not self.USING_COLAB:
            # En local, validation immédiate
            if not self.check_files_exist():
                missing = []
                if not self.video_path.exists():
                    missing.append(f"vidéo: {self.video_path}")
                if not self.config_path.exists():
                    missing.append(f"config: {self.config_path}")
                raise FileNotFoundError(f"❌ Fichiers manquants: {', '.join(missing)}")
            return True
        
        # Sur Colab, attente avec timeout
        print(f"⏳ Attente des fichiers (max {max_wait_minutes}min)...")
        start_time = time.time()
        max_wait_seconds = max_wait_minutes * 60
        
        while time.time() - start_time < max_wait_seconds:
            if self.check_files_exist():
                print("✅ Tous les fichiers sont disponibles!")
                return True
            
            elapsed = int(time.time() - start_time)
            remaining = max_wait_seconds - elapsed
            print(f"   ⏳ Attente... ({elapsed}s écoulées, {remaining}s restantes)")
            time.sleep(check_interval)
        
        print(f"⚠️ Timeout atteint ({max_wait_minutes}min)")
        return False
    
    def validate_files_now(self):
        """Validation immédiate avec erreur si fichiers manquants"""
        if not self.video_path.exists():
            raise FileNotFoundError(f"❌ Vidéo non trouvée: {self.video_path}")
        if not self.config_path.exists():
            raise FileNotFoundError(f"❌ Fichier config non trouvé: {self.config_path}")
        print("✅ Tous les fichiers sont valides")
    
    def display_config(self):
        """Affiche la configuration actuelle"""
        print(f"📋 CONFIGURATION CENTRALISÉE:")
        print(f"   🌍 Environnement: {'🔬 Colab' if self.USING_COLAB else '🖥️ Local'}")
        print(f"   🎬 Vidéo: {self.VIDEO_NAME}")
        print(f"   ⏯️  Intervalle frames: {self.FRAME_INTERVAL}")
        print(f"   🎬 Extraction: {'✅ Activée' if self.EXTRACT_FRAMES else '❌ Désactivée'}")
        print(f"   🔄 Force extraction: {'✅ Oui' if self.FORCE_EXTRACTION else '❌ Non'}")
        print(f"   🤖 Modèle SAM2: {self.SAM2_MODEL}")
        print(f"   📁 Dossier vidéos: {self.videos_dir}")
        print(f"   📄 Fichier config: {self.config_path}")
        print(f"   📁 Sortie: {self.output_dir}")
        print(f"   💾 Checkpoint: {self.checkpoint_path}")

# =============================================================================
# 🚀 INITIALISATION
# =============================================================================

# Création de l'instance de configuration
cfg = Config()

# Setup automatique des dossiers (toujours safe)
cfg.setup_directories()
cfg.display_config()

# Vérification des fichiers selon l'environnement
if cfg.USING_COLAB:
    print(f"\n🔬 Mode Colab détecté:")
    print(f"   💡 Les dossiers sont créés, vous pouvez maintenant uploader vos fichiers")
    print(f"   📤 Uploadez dans: {cfg.videos_dir}")
    print(f"   📄 Fichiers attendus:")
    print(f"      • {cfg.video_path.name}")
    print(f"      • {cfg.config_path.name}")
    
    # Vérification simple sans erreur
    cfg.check_files_exist()
    print(f"   💡 Utilisez cfg.wait_for_files() quand les uploads sont terminés")
else:
    print(f"\n🖥️ Mode Local détecté:")
    # Validation immédiate en local
    cfg.validate_files_now()

print("\n✅ Configuration centralisée initialisée!")

In [ ]:
# =============================================================================
# 🔧 INSTALLATION ET SETUP SAM2 (AUTO-INSTALL COLAB)
# =============================================================================

def is_sam2_installed():
    """Vérifie si SAM 2 est déjà installé"""
    try:
        import sam2
        return True
    except ImportError:
        return False

def install_sam2_colab(cfg):
    """Installation complète de SAM2 sur Colab avec chemins de la config"""
    print("🔧 Installation de SAM 2 en cours...")
    
    # Packages de base
    print("   📦 Installation des dépendances...")
    !{sys.executable} -m pip install opencv-python matplotlib
    !{sys.executable} -m pip install 'git+https://github.com/facebookresearch/sam2.git'
    
    # Création du dossier checkpoints (utilise la config)
    checkpoint_dir = cfg.checkpoint_path.parent
    print(f"   📁 Création du dossier checkpoints: {checkpoint_dir}")
    checkpoint_dir.mkdir(parents=True, exist_ok=True)
    
    # Téléchargement du modèle configuré
    checkpoint_url = "https://dl.fbaipublicfiles.com/segment_anything_2/092824"
    model_file = cfg.SAM2_CHECKPOINT
    
    print(f"   ⬇️ Téléchargement du modèle: {model_file}")
    !wget -P {checkpoint_dir} -q {checkpoint_url}/{model_file}
    
    # Optionnel: télécharger l'autre modèle si besoin
    if "large" in model_file:
        other_model = "sam2.1_hiera_small.pt"
        print(f"   📦 Modèle small également disponible: {other_model}")
        # !wget -P {checkpoint_dir} -q {checkpoint_url}/{other_model}
    else:
        other_model = "sam2.1_hiera_large.pt"
        print(f"   📦 Modèle large également disponible: {other_model}")
        # !wget -P {checkpoint_dir} -q {checkpoint_url}/{other_model}
    
    # Nettoyage Colab
    if cfg.USING_COLAB:
        print("   🧹 Nettoyage des fichiers temporaires Colab...")
        !rm -rf /content/sample_data/* 2>/dev/null || true
    
    # Nettoyage mémoire
    import gc
    import torch
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        print(f"   🔥 Mémoire GPU nettoyée")
    
    print("✅ Installation de SAM 2 terminée")

# =============================================================================
# 🚀 AUTO-SETUP SAM2 INTELLIGENT
# =============================================================================

print("🤖 Vérification et setup SAM2...")

# Vérifications
sam2_installed = is_sam2_installed()
checkpoint_available = cfg.checkpoint_path.exists()

print(f"   📦 SAM2 installé: {'✅' if sam2_installed else '❌'}")
print(f"   💾 Checkpoint disponible: {'✅' if checkpoint_available else '❌'} {cfg.checkpoint_path}")

# Logique d'installation
if cfg.USING_COLAB and (not sam2_installed or not checkpoint_available):
    print(f"🔬 Colab détecté - Installation automatique...")
    install_sam2_colab(cfg)
    
elif cfg.USING_COLAB and sam2_installed and checkpoint_available:
    print("✅ SAM 2 déjà installé sur Colab - SKIP installation")
    
elif not cfg.USING_COLAB:
    print("🖥️ Mode local détecté")
    if not sam2_installed:
        print("   ⚠️ SAM2 non installé. Installez avec: pip install git+https://github.com/facebookresearch/sam2.git")
    if not checkpoint_available:
        print(f"   ⚠️ Checkpoint manquant: {cfg.checkpoint_path}")
        print("   💡 Téléchargez depuis: https://github.com/facebookresearch/sam2#download-checkpoints")
    
    if sam2_installed and checkpoint_available:
        print("✅ SAM2 prêt en mode local")

print("✅ Setup SAM2 terminé")


In [ ]:
# =============================================================================
# 📦 IMPORTS ET CONFIGURATION ENVIRONNEMENT
# =============================================================================

# ==================== IMPORTS SYSTÈME ====================
import sys
import os
import json
import time
from pathlib import Path
from datetime import datetime
from typing import List, Tuple, Dict, Any, Optional
from dataclasses import dataclass
import uuid
import base64

# ==================== IMPORTS SCIENTIFIQUES ====================
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt

# ==================== IMPORTS DEEP LEARNING ====================
import torch
import torchvision
from pycocotools.mask import encode as encode_rle

# =============================================================================
# 🖥️ CONFIGURATION AUTOMATIQUE DU DEVICE ET OPTIMISATIONS
# =============================================================================

def setup_torch_environment(verbose=True):
    """Configure automatiquement l'environnement PyTorch optimal"""
    
    if verbose:
        print("🔧 Configuration de l'environnement PyTorch...")
        print(f"   📦 PyTorch version: {torch.__version__}")
        print(f"   📦 Torchvision version: {torchvision.__version__}")
    
    # === SÉLECTION DU DEVICE ===
    if torch.cuda.is_available():
        device = torch.device("cuda")
        gpu_name = torch.cuda.get_device_name(0)
        gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
        if verbose:
            print(f"   🔥 CUDA disponible: {gpu_name} ({gpu_memory:.1f}GB)")
    elif torch.backends.mps.is_available():
        device = torch.device("mps")
        if verbose:
            print(f"   🍎 MPS (Apple Silicon) disponible")
    else:
        device = torch.device("cpu")
        if verbose:
            print(f"   💻 CPU seulement")
    
    # === OPTIMISATIONS CUDA ===
    optimizations_applied = []
    
    if device.type == "cuda":
        # Autocast pour économiser la mémoire
        torch.autocast("cuda", dtype=torch.bfloat16).__enter__()
        optimizations_applied.append("Autocast bfloat16")
        
        # Optimisations TensorFloat-32 (si GPU récent)
        gpu_compute_capability = torch.cuda.get_device_properties(0).major
        if gpu_compute_capability >= 8:  # Ampere et plus récent
            torch.backends.cuda.matmul.allow_tf32 = True
            torch.backends.cudnn.allow_tf32 = True
            optimizations_applied.append("TensorFloat-32")
        
        # Optimisations mémoire additionnelles
        torch.backends.cudnn.benchmark = True  # Optimise pour tailles fixes
        optimizations_applied.append("cuDNN benchmark")
    
    if verbose and optimizations_applied:
        print(f"   ⚡ Optimisations activées: {', '.join(optimizations_applied)}")
    
    return device, optimizations_applied

def display_system_info(device):
    """Affiche les informations système détaillées"""
    print(f"\n📊 INFORMATIONS SYSTÈME:")
    print(f"   🖥️  Device principal: {device}")
    
    if device.type == "cuda":
        print(f"   🔥 GPU: {torch.cuda.get_device_name(0)}")
        print(f"   💾 Mémoire GPU: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f}GB")
        print(f"   🧮 Compute Capability: {torch.cuda.get_device_properties(0).major}.{torch.cuda.get_device_properties(0).minor}")
    
    # CPU Info
    import platform
    print(f"   💻 CPU: {platform.processor()}")
    print(f"   🧠 Threads: {torch.get_num_threads()}")
    
    # Versions importantes
    print(f"   🐍 Python: {platform.python_version()}")
    print(f"   📦 NumPy: {np.__version__}")
    print(f"   🎥 OpenCV: {cv2.__version__}")

# =============================================================================
# 🚀 INITIALISATION ENVIRONNEMENT
# =============================================================================

# Configuration automatique
device, optimizations = setup_torch_environment(verbose=True)

# Ajout du device à notre configuration centralisée
if 'cfg' in globals():
    cfg.device = device
    cfg.torch_optimizations = optimizations
    print(f"✅ Device ajouté à la configuration: cfg.device = {device}")
else:
    print(f"⚠️ Configuration cfg non trouvée - device disponible en tant que variable globale")

# Affichage des informations système (optionnel, décommentez si besoin)
# display_system_info(device)

# Nettoyage initial de la mémoire
if device.type == "cuda":
    torch.cuda.empty_cache()

print(f"\n✅ Environnement PyTorch configuré et optimisé!")

In [ ]:
# =============================================================================
# 📄 CHARGEMENT ET VALIDATION DE LA CONFIGURATION JSON
# =============================================================================

def load_and_validate_project_config(config_path: Path) -> Dict[str, Any]:
    """Charge et valide le fichier de configuration JSON du projet."""
    print(f"📄 Chargement de la configuration projet: {config_path}")

    # Vérification existence du fichier
    if not config_path.exists():
        raise FileNotFoundError(f"❌ Fichier config non trouvé: {config_path}")

    try:
        with open(config_path, 'r', encoding='utf-8') as f:
            config = json.load(f)
    except json.JSONDecodeError as e:
        raise ValueError(f"❌ Erreur JSON dans {config_path}: {e}")

    # === VALIDATION DE LA STRUCTURE ===
    required_sections = ['calibration', 'objects', 'initial_annotations']
    missing_sections = [section for section in required_sections if section not in config]
    
    if missing_sections:
        raise ValueError(f"❌ Sections manquantes dans le config: {missing_sections}")

    # === VALIDATION DES DONNÉES ===
    
    # Validation calibration
    if 'camera_parameters' not in config['calibration']:
        raise ValueError("❌ 'camera_parameters' manquant dans la calibration")
    
    # Validation objets
    if not config['objects']:
        raise ValueError("❌ Aucun objet défini dans la configuration")
    
    # Validation annotations initiales
    if not config['initial_annotations']:
        raise ValueError("❌ Aucune annotation initiale définie")

    # === STATISTIQUES ET RÉSUMÉ ===
    num_objects = len(config['objects'])
    
    # Comptage des annotations
    total_annotations = 0
    annotation_frames = set()
    for frame_data in config['initial_annotations']:
        frame_idx = frame_data.get('frame')
        annotations = frame_data.get('annotations', [])
        total_annotations += len(annotations)
        annotation_frames.add(frame_idx)

    # Types d'objets
    obj_types = {}
    obj_by_type = {}
    for obj in config['objects']:
        obj_type = obj.get('obj_type', 'unknown')
        obj_types[obj_type] = obj_types.get(obj_type, 0) + 1
        if obj_type not in obj_by_type:
            obj_by_type[obj_type] = []
        obj_by_type[obj_type].append(obj.get('obj_id', 'no_id'))

    print(f"✅ Configuration projet chargée et validée:")
    print(f"   📷 Calibration caméra: ✅ OK")
    print(f"   🎯 Objets définis: {num_objects}")
    print(f"   📍 Annotations initiales: {total_annotations} sur {len(annotation_frames)} frames")
    print(f"   🏷️  Types d'objets:")
    for obj_type, count in obj_types.items():
        ids = obj_by_type[obj_type]
        print(f"      • {obj_type}: {count} ({ids})")

    return config

# =============================================================================
# 🚀 CHARGEMENT INTELLIGENT DE LA CONFIGURATION
# =============================================================================

# Chargement avec gestion intelligente Colab/Local
if cfg.USING_COLAB:
    print("🔬 Mode Colab détecté pour le chargement config")
    if cfg.config_path.exists():
        project_config = load_and_validate_project_config(cfg.config_path)
    else:
        print(f"⏳ Fichier config pas encore uploadé, utilisez:")
        print(f"   project_config = wait_and_load_config(cfg)")
        project_config = None
else:
    print("🖥️ Mode local: chargement immédiat")
    project_config = load_and_validate_project_config(cfg.config_path)

if project_config:
    print("\n✅ Configuration projet prête à l'usage!")

In [ ]:
# =============================================================================
# 🎬 EXTRACTION DES FRAMES
# =============================================================================

def extract_frames(video_path: Path, frames_dir: Path, frame_interval: int = 1, force_extraction: bool = False) -> int:
    """Extrait les frames de la vidéo selon l'intervalle spécifié."""

    print(f"🎬 Extraction des frames...")
    print(f"   📹 Source: {video_path}")
    print(f"   📁 Destination: {frames_dir}")
    print(f"   ⏯️  Intervalle: {frame_interval}")
    print(f"   🔄 Force extraction: {'✅ Oui' if force_extraction else '❌ Non'}")

    # Vérification existence du fichier vidéo
    if not video_path.exists():
        raise FileNotFoundError(f"❌ Vidéo non trouvée: {video_path}")

    # Vérification si extraction déjà faite
    existing_frames = list(frames_dir.glob("*.jpg"))
    if existing_frames and not force_extraction:
        print(f"📂 {len(existing_frames)} frames déjà extraites - SKIP")
        return len(existing_frames)
    elif existing_frames and force_extraction:
        print(f"🔄 {len(existing_frames)} frames existantes - SUPPRESSION et ré-extraction...")
        # Supprimer les frames existantes
        for frame_file in existing_frames:
            frame_file.unlink()
        print(f"🗑️  Frames existantes supprimées")

    cap = cv2.VideoCapture(str(video_path))
    if not cap.isOpened():
        raise ValueError(f"❌ Impossible d'ouvrir la vidéo: {video_path}")

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    print(f"📊 Vidéo: {total_frames} frames, {fps:.1f} FPS")
    print(f"📊 Frames à extraire: ~{total_frames // frame_interval}")

    extracted_count = 0
    frame_idx = 0

    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break

            # Extraire seulement selon l'intervalle
            if frame_idx % frame_interval == 0:
                output_idx = frame_idx // frame_interval
                filename = frames_dir / f"{output_idx:05d}.jpg"
                cv2.imwrite(str(filename), frame, [cv2.IMWRITE_JPEG_QUALITY, 95])
                extracted_count += 1

                if extracted_count % 50 == 0:
                    progress = (frame_idx / total_frames) * 100
                    print(f"📊 Progrès: {extracted_count} frames extraites ({progress:.1f}%)")

            frame_idx += 1

    finally:
        cap.release()

    print(f"✅ {extracted_count} frames extraites")
    return extracted_count

def count_existing_frames(frames_dir: Path) -> int:
    """Compte les frames existantes dans le dossier"""
    existing_frames = list(frames_dir.glob("*.jpg"))
    return len(existing_frames)

def extract_frames_with_config(cfg) -> int:
    """Extrait les frames en utilisant la configuration centralisée"""
    
    # Vérification des fichiers (gestion Colab/Local)
    if cfg.USING_COLAB and not cfg.video_path.exists():
        print("🔬 Mode Colab: Vidéo pas encore uploadée")
        print(f"   💡 Uploadez {cfg.video_path.name} dans {cfg.videos_dir}")
        print(f"   💡 Puis relancez cette cellule ou utilisez cfg.wait_for_files()")
        return 0
    
    # Extraction selon configuration
    if cfg.EXTRACT_FRAMES:
        return extract_frames(
            video_path=cfg.video_path,
            frames_dir=cfg.frames_dir,
            frame_interval=cfg.FRAME_INTERVAL,
            force_extraction=cfg.FORCE_EXTRACTION
        )
    else:
        # Skip extraction - compter les frames existantes
        existing_count = count_existing_frames(cfg.frames_dir)
        
        print(f"⏭️  Extraction désactivée (cfg.EXTRACT_FRAMES = False)")
        if existing_count > 0:
            print(f"📂 Utilisation de {existing_count} frames existantes")
        else:
            print(f"⚠️  Aucune frame trouvée dans {cfg.frames_dir}")
            print(f"💡 Conseil: Activez cfg.EXTRACT_FRAMES = True pour extraire")
        
        return existing_count

# =============================================================================
# 🚀 EXTRACTION AVEC CONFIGURATION CENTRALISÉE
# =============================================================================

# Extraction intelligente selon l'environnement
print("🎬 Démarrage de l'extraction des frames...")

# Vérification prérequis selon l'environnement
if cfg.USING_COLAB:
    print("🔬 Mode Colab: Vérification des fichiers uploadés...")
    if not cfg.video_path.exists():
        print(f"⚠️  Vidéo pas encore uploadée: {cfg.video_path.name}")
        print(f"   📤 Uploadez dans: {cfg.videos_dir}")
        print(f"   🔄 Puis relancez cette cellule")
        extracted_frames_count = 0
    else:
        extracted_frames_count = extract_frames_with_config(cfg)
else:
    print("🖥️ Mode Local: Extraction directe")
    extracted_frames_count = extract_frames_with_config(cfg)

# Ajout du résultat à la configuration pour usage ultérieur
cfg.extracted_frames_count = extracted_frames_count

print(f"\n📊 RÉSUMÉ EXTRACTION:")
print(f"   🎬 Frames extraites/comptées: {extracted_frames_count}")
print(f"   📁 Dossier: {cfg.frames_dir}")
print(f"   ✅ cfg.extracted_frames_count = {extracted_frames_count}")

In [ ]:
# =============================================================================
# 🤖 INITIALISATION SAM2
# =============================================================================

def initialize_sam2_predictor(cfg, verbose=True):
    """Initialise le predictor SAM2 avec la configuration centralisée"""
    
    # Import SAM2
    from sam2.build_sam import build_sam2_video_predictor
    
    if verbose:
        print(f"🤖 Initialisation SAM2...")
        print(f"   🧠 Modèle: {cfg.model_config_path}")
        print(f"   💾 Checkpoint: {cfg.checkpoint_path}")
        print(f"   🖥️  Device: {cfg.device}")
    
    # Vérification du checkpoint
    if not cfg.checkpoint_path.exists():
        raise FileNotFoundError(f"❌ Checkpoint SAM2 non trouvé: {cfg.checkpoint_path}")
    
    # Construction du predictor
    predictor = build_sam2_video_predictor(
        config_file=cfg.model_config_path,
        ckpt_path=str(cfg.checkpoint_path),
        device=cfg.device
    )
    
    return predictor

def initialize_inference_state(predictor, cfg, verbose=True):
    """Initialise l'état d'inférence avec vérifications"""
    
    if verbose:
        print(f"\n🎬 Initialisation état d'inférence...")
        print(f"   📁 Frames: {cfg.frames_dir}")
    
    # Vérification des frames
    if cfg.extracted_frames_count == 0:
        raise ValueError(f"❌ Aucune frame extraite. Extrayez d'abord les frames.")
    
    # Initialisation de l'état d'inférence
    inference_state = predictor.init_state(
        video_path=str(cfg.frames_dir),
        offload_video_to_cpu=True,    # Économise la mémoire GPU
        offload_state_to_cpu=False    # Garde l'état en GPU
    )
    
    # Reset de l'état
    predictor.reset_state(inference_state)
    
    # Vérification
    loaded_frames = inference_state["num_frames"]
    
    if verbose:
        print(f"\n✅ SAM2 initialisé:")
        print(f"   🖼️  Frames extraites: {cfg.extracted_frames_count}")
        print(f"   🎬 Frames chargées: {loaded_frames}")
        print(f"   ✅ Correspondance: {'OK' if cfg.extracted_frames_count == loaded_frames else 'ERREUR'}")
        
        if cfg.device.type == "cuda":
            allocated = torch.cuda.memory_allocated() / 1024**3
            print(f"   💾 GPU Memory: {allocated:.2f}GB")
    
    if cfg.extracted_frames_count != loaded_frames:
        print(f"⚠️ Incohérence frames : {cfg.extracted_frames_count} extraites vs {loaded_frames} chargées")
    
    return inference_state

# =============================================================================
# 🚀 INITIALISATION AVEC CONFIGURATION CENTRALISÉE
# =============================================================================

# Vérification prérequis
if not hasattr(cfg, 'extracted_frames_count') or cfg.extracted_frames_count == 0:
    print("❌ Frames non extraites. Exécutez d'abord la cellule d'extraction des frames.")
    print("💡 Ou définissez cfg.extracted_frames_count manuellement si frames déjà présentes")
else:
    # Initialisation SAM2
    predictor = initialize_sam2_predictor(cfg)
    inference_state = initialize_inference_state(predictor, cfg)
    
    # Ajout à la configuration pour usage ultérieur
    cfg.predictor = predictor
    cfg.inference_state = inference_state
    
    print("\n✅ SAM2 prêt pour l'ajout d'annotations!")

In [ ]:
# =============================================================================
# 🎯 AJOUT DES ANNOTATIONS INITIALES
# =============================================================================

def add_initial_annotations(predictor, inference_state, project_config: Dict[str, Any]):
    """Ajoute les annotations initiales depuis la configuration projet."""

    print(f"🎯 Ajout des annotations initiales...")

    # Création du mapping obj_id -> obj_type
    obj_types = {}
    for obj in project_config['objects']:
        obj_types[obj['obj_id']] = obj['obj_type']

    # Extraction automatique des annotations et frames depuis le JSON
    all_annotations = []
    annotation_frames = []

    for frame_data in project_config['initial_annotations']:
        frame_idx = frame_data['frame']
        annotations = frame_data['annotations']
        annotation_frames.append(frame_idx)

        print(f"   📍 Frame {frame_idx}: {len(annotations)} annotations")

        for annotation in annotations:
            all_annotations.append({
                'frame': frame_idx,
                'obj_id': annotation['obj_id'],
                'points': annotation['points'],
                'obj_type': obj_types.get(annotation['obj_id'], f'unknown_{annotation["obj_id"]}')
            })

    if not all_annotations:
        raise ValueError(f"❌ Aucune annotation trouvée dans le fichier config")

    print(f"   📊 Total: {len(all_annotations)} annotations sur {len(set(annotation_frames))} frames")

    # Ajout des annotations à SAM2
    added_objects = []

    for annotation_data in all_annotations:
        frame_idx = annotation_data['frame']
        obj_id = annotation_data['obj_id']
        obj_type = annotation_data['obj_type']
        points_data = annotation_data['points']

        # Extraction des coordonnées et labels
        points = np.array([[p['x'], p['y']] for p in points_data], dtype=np.float32)
        labels = np.array([p['label'] for p in points_data], dtype=np.int32)

        print(f"   🎯 Frame {frame_idx} - Objet {obj_id} ({obj_type}): {len(points)} points à ({points[0][0]:.0f}, {points[0][1]:.0f})")

        # Ajout à SAM2 avec add_new_points_or_box
        _, out_obj_ids, out_mask_logits = predictor.add_new_points_or_box(
            inference_state,
            frame_idx,
            obj_id,
            points,
            labels
        )

        # Éviter les doublons dans added_objects
        if not any(obj['obj_id'] == obj_id for obj in added_objects):
            added_objects.append({
                'obj_id': obj_id,
                'obj_type': obj_type,
                'points_count': len(points)
            })

    # Vérification
    sam_obj_ids = inference_state["obj_ids"]

    print(f"\n📊 RÉSUMÉ ANNOTATIONS:")
    print(f"   🎯 Annotations configurées: {len(all_annotations)}")
    print(f"   🎯 Objets uniques: {len(added_objects)}")
    print(f"   ✅ Objets ajoutés à SAM2: {len(sam_obj_ids)}")
    print(f"   🆔 IDs: {sorted(sam_obj_ids)}")
    print(f"   📍 Frames utilisées: {sorted(set(annotation_frames))}")

    # Résumé par type
    type_counts = {}
    for obj in added_objects:
        obj_type = obj['obj_type']
        type_counts[obj_type] = type_counts.get(obj_type, 0) + 1
    print(f"   🏷️  Types: {dict(type_counts)}")

    return added_objects, all_annotations

# =============================================================================
# 🚀 AJOUT AVEC CONFIGURATION CENTRALISÉE
# =============================================================================

# Vérifications prérequis
if not hasattr(cfg, 'predictor') or not hasattr(cfg, 'inference_state'):
    print("❌ SAM2 non initialisé. Exécutez d'abord la cellule d'initialisation SAM2.")
elif project_config is None:
    print("❌ Configuration projet non chargée.")
    if cfg.USING_COLAB:
        print("💡 Sur Colab, utilisez: project_config = wait_and_load_config(cfg)")
    else:
        print("💡 La configuration devrait être chargée automatiquement")
else:
    # Ajout des annotations
    added_objects, initial_annotations_data = add_initial_annotations(
        cfg.predictor, 
        cfg.inference_state, 
        project_config
    )
    
    # Ajout à la configuration pour usage ultérieur
    cfg.added_objects = added_objects
    cfg.initial_annotations_data = initial_annotations_data
    
    print("\n✅ Annotations initiales ajoutées avec succès!")

In [ ]:
# =============================================================================
# 🔧 FONCTIONS UTILITAIRES POUR ANNOTATIONS COMPLÈTES
# =============================================================================

def get_object_scores(predictor, inference_state, frame_idx, obj_id):
    """Récupère les scores d'objet de manière propre et sûre"""
    try:
        obj_idx = predictor._obj_id_to_idx(inference_state, obj_id)
        obj_output_dict = inference_state["output_dict_per_obj"][obj_idx]
        temp_output_dict = inference_state["temp_output_dict_per_obj"][obj_idx]

        # Chercher dans les outputs
        frame_output = None
        if frame_idx in temp_output_dict["cond_frame_outputs"]:
            frame_output = temp_output_dict["cond_frame_outputs"][frame_idx]
        elif frame_idx in temp_output_dict["non_cond_frame_outputs"]:
            frame_output = temp_output_dict["non_cond_frame_outputs"][frame_idx]
        elif frame_idx in obj_output_dict["cond_frame_outputs"]:
            frame_output = obj_output_dict["cond_frame_outputs"][frame_idx]
        elif frame_idx in obj_output_dict["non_cond_frame_outputs"]:
            frame_output = obj_output_dict["non_cond_frame_outputs"][frame_idx]

        if frame_output and "object_score_logits" in frame_output:
            object_score_logits = frame_output["object_score_logits"]
            return torch.sigmoid(object_score_logits).item()
        return None

    except Exception as e:
        return None

def calculate_bbox_from_rle(rle_data: Dict[str, Any]) -> Optional[Dict[str, int]]:
    """Calcule la bounding box depuis un RLE base64."""
    from pycocotools.mask import toBbox

    try:
        rle = {
            "size": rle_data["size"],
            "counts": base64.b64decode(rle_data["counts"])
        }

        bbox = toBbox(rle)

        result = {
            "x": int(bbox[0]),
            "y": int(bbox[1]),
            "width": int(bbox[2]),
            "height": int(bbox[3])
        }
        return result

    except Exception as e:
        print(f"⚠️ Erreur calcul bbox: {e}")
        return None

def image_to_world(point_2d, cam_params):
    """
    Projette un point 2D de l'image vers le plan du terrain (Z=0).
    """
    # Create projection matrix P
    K = np.array([
        [cam_params["cam_params"]["x_focal_length"], 0, cam_params["cam_params"]["principal_point"][0]],
        [0, cam_params["cam_params"]["y_focal_length"], cam_params["cam_params"]["principal_point"][1]],
        [0, 0, 1]
    ])
    R = np.array(cam_params["cam_params"]["rotation_matrix"])
    t = -R @ np.array(cam_params["cam_params"]["position_meters"])
    P = K @ np.hstack((R, t.reshape(-1,1)))

    # Create point on image plane in homogeneous coordinates
    point_2d_h = np.array([point_2d[0], point_2d[1], 1])

    # Back-project ray from camera
    ray = np.linalg.inv(K) @ point_2d_h
    ray = R.T @ ray

    # Find intersection with Z=0 plane
    camera_pos = np.array(cam_params["cam_params"]["position_meters"])
    t = -camera_pos[2] / ray[2]
    world_point = camera_pos + t * ray

    return world_point[:2]  # Return only X,Y coordinates since Z=0

def calculate_points_output(bbox_output: dict, cam_params: dict = None) -> dict:
    """
    Calcule les points de sortie à partir de la bbox output.

    Args:
        bbox_output: Dict avec 'x', 'y', 'width', 'height'
        cam_params: Paramètres de calibration caméra pour projection terrain

    Returns:
        Dict avec les points calculés séparés par plan (image vs field)
    """
    if not bbox_output:
        return None

    # Calculer le point CENTER_BOTTOM dans le plan image
    center_bottom_x = bbox_output['x'] + bbox_output['width'] / 2
    center_bottom_y = bbox_output['y'] + bbox_output['height']  # Bas de la bbox

    # Structure avec séparation image/field
    points_output = {
        "image": {
            "CENTER_BOTTOM": {
                "x": float(center_bottom_x),
                "y": float(center_bottom_y)
            }
        },
        "field": {
            "CENTER_BOTTOM": None
        }
    }

    # Projection vers le terrain si les paramètres caméra sont fournis
    if cam_params:
        try:
            # Projeter le point CENTER_BOTTOM vers le terrain
            image_point = [center_bottom_x, center_bottom_y]
            field_point = image_to_world(image_point, cam_params)

            points_output["field"]["CENTER_BOTTOM"] = {
                "x": float(field_point[0]),
                "y": float(field_point[1])
            }

        except Exception as e:
            print(f"⚠️ Erreur projection terrain: {e}")
            points_output["field"]["CENTER_BOTTOM"] = None

    return points_output

def create_mask_annotation(obj_id: int, mask_logits, predictor=None, inference_state=None,
                         frame_idx=None, cam_params: Dict = None) -> Dict:
    """
    Crée une annotation de masque complète avec points input/output, bbox et scores.
    """
    # Conversion en masque binaire
    mask = (mask_logits > 0.0).cpu().numpy()
    if mask.ndim == 3 and mask.shape[0] == 1:
        mask = np.squeeze(mask, axis=0)

    # Encodage RLE
    if not mask.flags['F_CONTIGUOUS']:
        mask = np.asfortranarray(mask)

    rle = encode_rle(mask.astype(np.uint8))
    base64_counts = base64.b64encode(rle["counts"]).decode('ascii')

    # Calcul bbox et points output si masque non vide
    bbox_output = None
    points_output = None

    if mask.sum() > 0:
        from pycocotools.mask import toBbox
        bbox = toBbox(rle)
        bbox_output = {
            "x": int(bbox[0]),
            "y": int(bbox[1]),
            "width": int(bbox[2]),
            "height": int(bbox[3])
        }
        # Calcul des points output depuis la bbox
        points_output = calculate_points_output(bbox_output, cam_params)

    # Récupération du score du masque
    mask_score = None

    if predictor and inference_state and frame_idx is not None:
        # Score du masque
        mask_score = get_object_scores(predictor, inference_state, frame_idx, obj_id)

    # Structure d'annotation complète
    return {
        "id": str(uuid.uuid4()),
        "objectId": str(obj_id),
        "type": "mask",
        "mask": {
            "format": "rle_coco_base64",
            "size": [int(rle["size"][0]), int(rle["size"][1])],
            "counts": base64_counts
        },
        "bbox": {
            "output": bbox_output
        },
        "points": {
            "output": points_output
        },
        "maskScore": mask_score,
        "pose": None,
        "warning": False
    }

print("✅ Fonctions utilitaires pour annotations chargées!")

In [ ]:
# =============================================================================
# 🔄 PROPAGATION ET GÉNÉRATION DES ANNOTATIONS
# =============================================================================

def generate_frame_mapping(total_frames: int, frame_interval: int) -> List[Optional[int]]:
    """
    Génère le mapping entre frames originales et frames traitées.
    
    Args:
        total_frames: Nombre total de frames dans la vidéo originale
        frame_interval: Intervalle entre frames (ex: 10 = 1 frame sur 10)
    
    Returns:
        Liste où l'index = frame originale, valeur = frame traitée (ou None si pas traitée)
    """
    frame_mapping = [None] * total_frames
    processed_idx = 0

    for original_idx in range(total_frames):
        if original_idx % frame_interval == 0:
            frame_mapping[original_idx] = processed_idx
            processed_idx += 1

    return frame_mapping

def get_video_info(video_path: Path) -> Dict[str, Any]:
    """Récupère les informations de la vidéo"""
    cap = cv2.VideoCapture(str(video_path))
    if not cap.isOpened():
        raise ValueError(f"❌ Impossible d'ouvrir la vidéo: {video_path}")
    
    video_info = {
        'total_frames': int(cap.get(cv2.CAP_PROP_FRAME_COUNT)),
        'fps': cap.get(cv2.CAP_PROP_FPS),
        'width': int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
        'height': int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    }
    cap.release()
    return video_info

def create_project_structure_with_config(cfg, project_config: Dict[str, Any], added_objects: List[Dict]) -> Dict[str, Any]:
    """Crée la structure JSON du projet avec la configuration centralisée."""
    
    # Informations vidéo
    video_info = get_video_info(cfg.video_path)
    
    # Calcul du frame mapping
    frame_mapping = generate_frame_mapping(video_info['total_frames'], cfg.FRAME_INTERVAL)
    processed_frame_count = sum(x is not None for x in frame_mapping)

    # Structure objects avec couleurs
    import random
    import colorsys

    config_objects_mapping = {}
    for obj in project_config['objects']:
        config_objects_mapping[obj['obj_id']] = obj

    objects = {}
    for obj_data in added_objects:
        obj_id = str(obj_data['obj_id'])
        obj_type = obj_data['obj_type']

        # Récupérer les informations complètes depuis le config
        config_obj = config_objects_mapping.get(int(obj_id), {})

        # Couleur aléatoire reproductible
        random.seed(int(obj_id) * 12345)
        hue = random.random()
        rgb = colorsys.hsv_to_rgb(hue, 0.8, 0.9)
        hex_color = "#{:02x}{:02x}{:02x}".format(
            int(rgb[0] * 255), int(rgb[1] * 255), int(rgb[2] * 255)
        )

        objects[obj_id] = {
            "id": obj_id,
            "type": obj_type,
            "team": config_obj.get('team', None),
            "jersey_number": config_obj.get('jersey_number', None),
            "jersey_color": config_obj.get('jersey_color', None),
            "role": config_obj.get('role', None),
            "display_color": hex_color
        }

    return {
        "format_version": "1.0",
        "video": f"{cfg.VIDEO_NAME}.mp4",
        "metadata": {
            "project_id": str(uuid.uuid4()),
            "created_at": datetime.now().isoformat() + "Z",
            "fps": video_info['fps'],
            "resolution": {
                "width": video_info['width'],
                "height": video_info['height'],
                "aspect_ratio": round(video_info['width'] / video_info['height'], 2)
            },
            "frame_interval": cfg.FRAME_INTERVAL,
            "frame_count_original": video_info['total_frames'],
            "frame_count_processed": processed_frame_count,
            "frame_mapping": frame_mapping,
            "static_video": False
        },
        "calibration": project_config['calibration'],
        "objects": objects,
        "initial_annotations": project_config['initial_annotations'],
        "annotations": {}
    }

def run_sam2_propagation(cfg, project_config):
    """Exécute la propagation SAM2 et génère toutes les annotations"""
    
    print(f"🔄 Démarrage de la propagation...")
    print(f"   🎬 {cfg.extracted_frames_count} frames à traiter")
    print(f"   🎯 {len(cfg.added_objects)} objets à suivre")

    # Création de la structure du projet
    project = create_project_structure_with_config(cfg, project_config, cfg.added_objects)

    # Propagation et annotation
    frame_count = 0
    for out_frame_idx, out_obj_ids, out_mask_logits in cfg.predictor.propagate_in_video(cfg.inference_state):

        if str(out_frame_idx) not in project['annotations']:
            project['annotations'][str(out_frame_idx)] = []

        for i, out_obj_id in enumerate(out_obj_ids):
            annotation = create_mask_annotation(
                obj_id=out_obj_id,
                mask_logits=out_mask_logits[i],
                predictor=cfg.predictor,
                inference_state=cfg.inference_state,
                frame_idx=out_frame_idx,
                cam_params=project_config['calibration']['camera_parameters']
            )
            project['annotations'][str(out_frame_idx)].append(annotation)
        
        frame_count += 1
        if frame_count % 50 == 0:
            progress = (frame_count / cfg.extracted_frames_count) * 100
            print(f"   📊 Progrès: {frame_count}/{cfg.extracted_frames_count} frames ({progress:.1f}%)")
    
    print(f"✅ Propagation terminée: {frame_count} frames traitées")
    return project

# =============================================================================
# 🚀 EXÉCUTION DE LA PROPAGATION
# =============================================================================

# Vérifications des prérequis
missing_requirements = []
required_attrs = ['predictor', 'inference_state', 'added_objects', 'extracted_frames_count']

for attr in required_attrs:
    if not hasattr(cfg, attr):
        missing_requirements.append(attr)

if project_config is None:
    missing_requirements.append("project_config")

if missing_requirements:
    print(f"❌ Prérequis manquants: {missing_requirements}")
    print("💡 Exécutez d'abord les cellules précédentes dans l'ordre:")
    print("   1. Configuration centralisée")
    print("   2. Installation SAM2") 
    print("   3. Chargement configuration projet")
    print("   4. Extraction des frames")
    print("   5. Initialisation SAM2")
    print("   6. Ajout des annotations initiales")
else:
    # Exécution de la propagation
    project = run_sam2_propagation(cfg, project_config)
    
    # Ajout à la configuration pour usage ultérieur
    cfg.project = project
    
    # Statistiques finales
    total_annotations = sum(len(annotations) for annotations in project['annotations'].values())
    unique_frames = len(project['annotations'])
    
    print(f"\n📊 RÉSULTATS PROPAGATION:")
    print(f"   🎬 Frames originales: {project['metadata']['frame_count_original']}")
    print(f"   🎬 Frames traitées: {project['metadata']['frame_count_processed']}")
    print(f"   📍 Frames avec annotations: {unique_frames}")
    print(f"   📍 Annotations totales: {total_annotations}")
    print(f"   🎯 Objets suivis: {len(project['objects'])}")
    print(f"   ⏯️  Intervalle frames: {project['metadata']['frame_interval']}")
    
    print("\n✅ Propagation terminée avec succès!")

In [ ]:
# =============================================================================
# 💾 SAUVEGARDE DES RÉSULTATS
# =============================================================================

def save_project_results(cfg, project, verbose=True):
    """Sauvegarde les résultats du projet avec gestion d'erreurs"""
    
    if verbose:
        print(f"💾 Sauvegarde des résultats...")
        print(f"   📄 Fichier JSON: {cfg.output_json_path}")
    
    try:
        # Sauvegarde du JSON principal
        with open(cfg.output_json_path, 'w', encoding='utf-8') as f:
            json.dump(project, f, indent=2, ensure_ascii=False)
        
        if verbose:
            print(f"✅ Fichier JSON sauvé: {cfg.output_json_path}")
            
        # Sauvegarde d'une version compacte (sans indent pour économiser l'espace)
        compact_json_path = cfg.output_dir / f"{cfg.VIDEO_NAME}_project_compact.json"
        with open(compact_json_path, 'w', encoding='utf-8') as f:
            json.dump(project, f, separators=(',', ':'), ensure_ascii=False)
        
        if verbose:
            # Tailles des fichiers
            json_size = cfg.output_json_path.stat().st_size / 1024**2  # MB
            compact_size = compact_json_path.stat().st_size / 1024**2  # MB
            print(f"   📄 Version formatée: {json_size:.1f}MB")
            print(f"   📄 Version compacte: {compact_size:.1f}MB")
            
        return True
        
    except Exception as e:
        print(f"❌ Erreur lors de la sauvegarde: {e}")
        return False

def display_final_statistics(cfg, project):
    """Affiche les statistiques finales du projet"""
    
    # Calcul des statistiques
    total_annotations = sum(len(annotations) for annotations in project['annotations'].values())
    unique_frames = len(project['annotations'])
    
    # Calcul de la taille des données
    json_size = cfg.output_json_path.stat().st_size / 1024**2 if cfg.output_json_path.exists() else 0
    
    print(f"\n📊 RÉSULTATS FINAUX:")
    print(f"   🎬 Frames originales: {project['metadata']['frame_count_original']:,}")
    print(f"   🎬 Frames traitées: {project['metadata']['frame_count_processed']:,}")
    print(f"   📍 Frames avec annotations: {unique_frames:,}")
    print(f"   📍 Annotations totales: {total_annotations:,}")
    print(f"   🎯 Objets suivis: {len(project['objects'])}")
    print(f"   ⏯️  Intervalle frames: {project['metadata']['frame_interval']}")
    print(f"   📄 Taille JSON: {json_size:.1f}MB")
    print(f"   📁 Dossier de sortie: {cfg.output_dir}")
    
    # Affichage d'un échantillon du mapping
    sample_mapping = [(i, v) for i, v in enumerate(project['metadata']['frame_mapping'][:50]) if v is not None]
    print(f"   🗂️  Mapping échantillon (original→traité): {sample_mapping[:5]}...")
    
    # Résumé par type d'objet
    if project['objects']:
        type_counts = {}
        for obj_data in project['objects'].values():
            obj_type = obj_data.get('type', 'unknown')
            type_counts[obj_type] = type_counts.get(obj_type, 0) + 1
        print(f"   🏷️  Types d'objets: {dict(type_counts)}")
def create_colab_backup(cfg, use_timestamp=False, custom_suffix=""):
    """Crée une sauvegarde sur Google Drive pour Colab"""
    
    if not cfg.USING_COLAB:
        print("🖥️ Mode local: sauvegarde Google Drive skippée")
        return False
    
    try:
        from google.colab import drive
        import shutil
        from datetime import datetime
        
        print("🔬 Mode Colab: Création de la sauvegarde Google Drive...")
        
        # Monter le Drive
        print("   📁 Montage Google Drive...")
        drive.mount('/content/drive', force_remount=False)
        
        # Construction du nom selon les options
        if use_timestamp:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            zip_name = f"{cfg.VIDEO_NAME}_{timestamp}"
        elif custom_suffix:
            zip_name = f"{cfg.VIDEO_NAME}_{custom_suffix}"
        else:
            zip_name = cfg.VIDEO_NAME
        
        drive_zip_path = f'/content/drive/MyDrive/{zip_name}'
        
        # Vérifier si le fichier existe déjà
        existing_zip = Path(f"{drive_zip_path}.zip")
        if existing_zip.exists():
            existing_size = existing_zip.stat().st_size / 1024**2
            print(f"   🔄 Fichier existant trouvé: {zip_name}.zip ({existing_size:.1f}MB) - sera écrasé")
        
        print(f"   📦 Création du ZIP: {zip_name}.zip")
        
        # Créer le ZIP avec tout le dossier videos
        shutil.make_archive(drive_zip_path, 'zip', str(cfg.videos_dir))
        
        # Vérifier la taille
        zip_path = Path(f"{drive_zip_path}.zip")
        if zip_path.exists():
            zip_size = zip_path.stat().st_size / 1024**2  # MB
            print(f"✅ Sauvegarde créée: {zip_name}.zip ({zip_size:.1f}MB)")
            print(f"   📁 Emplacement: MyDrive/{zip_name}.zip")
            return True
        else:
            print("❌ Erreur: fichier ZIP non créé")
            return False
            
    except Exception as e:
        print(f"❌ Erreur sauvegarde Google Drive: {e}")
        return False

def cleanup_temporary_files(cfg):
    """Nettoie les fichiers temporaires (optionnel)"""
    
    print("🧹 Nettoyage optionnel...")
    
    # Comptage des frames
    frame_count = len(list(cfg.frames_dir.glob("*.jpg")))
    frame_size = sum(f.stat().st_size for f in cfg.frames_dir.glob("*.jpg")) / 1024**2
    
    print(f"   🖼️  Frames extraites: {frame_count} fichiers ({frame_size:.1f}MB)")
    print(f"   💡 Pour libérer l'espace, vous pouvez supprimer: {cfg.frames_dir}")
    print(f"   💡 Les frames peuvent être ré-extraites avec FORCE_EXTRACTION=True")

# =============================================================================
# 🚀 SAUVEGARDE AVEC ÉCRASEMENT PAR DÉFAUT
# =============================================================================

# Vérification des prérequis
if not hasattr(cfg, 'project') or cfg.project is None:
    print("❌ Projet non disponible. Exécutez d'abord la propagation.")
else:
    print(f"💾 Démarrage de la sauvegarde...")
    
    # Sauvegarde principale
    save_success = save_project_results(cfg, cfg.project)
    
    if save_success:
        # Affichage des statistiques
        display_final_statistics(cfg, cfg.project)
        
        # Sauvegarde Colab si applicable
        if cfg.USING_COLAB:
            print(f"\n🔬 Sauvegarde Google Drive...")
            
            # Mode par défaut : écrase l'ancienne version
            colab_success = create_colab_backup(cfg)  # use_timestamp=False par défaut
            
            # Si vous voulez un timestamp occasionnellement, décommentez :
            # colab_success = create_colab_backup(cfg, use_timestamp=True)
            
            if colab_success:
                print("✅ Sauvegarde Google Drive réussie!")
        
        # Nettoyage optionnel
        print(f"\n🧹 Informations de nettoyage:")
        cleanup_temporary_files(cfg)
        
        print(f"\n🎉 Pipeline SAM2 terminé avec succès!")
        print(f"📄 Fichier principal: {cfg.output_json_path}")
        if cfg.USING_COLAB:
            print(f"☁️ Sauvegarde Drive: MyDrive/{cfg.VIDEO_NAME}.zip")
            
    else:
        print("❌ Échec de la sauvegarde")